<a href="https://colab.research.google.com/github/Pavel-Zinkevich/Quizz-bot/blob/main/bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install aiogram

In [ ]:
!pip install aiosqlite

In [ ]:
# Установим библиотеку nest_asyncio
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
folder_path = "/content/drive/MyDrive/Colab Notebooks/Quizz_bot/"

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Quizz_bot')

In [ ]:
%run "/content/drive/MyDrive/Colab Notebooks/Quizz_bot/config.ipynb"
%run "/content/drive/MyDrive/Colab Notebooks/Quizz_bot/db.ipynb"
%run "/content/drive/MyDrive/Colab Notebooks/Quizz_bot/questions.ipynb"


In [ ]:
import aiosqlite

async def check_table_structure():
    async with aiosqlite.connect(DB_NAME) as db:
        # Выполнение PRAGMA для получения информации о таблице
        async with db.execute('PRAGMA table_info(quiz_state)') as cursor:
            columns = await cursor.fetchall()  # Получаем все данные из результата запроса
            if columns:
                print("Структура таблицы:")
                for column in columns:
                    print(f"Column ID: {column[0]}, Name: {column[1]}, Type: {column[2]}, Not Null: {column[3]}, Default: {column[4]}, Primary Key: {column[5]}")
            else:
                print("Таблица не существует или ошибка в названии таблицы.")
async def add_score_column():
    async with aiosqlite.connect(DB_NAME) as db:
        await db.execute('''
        ALTER TABLE quiz_state ADD COLUMN score INTEGER DEFAULT 0
        ''')
        await db.commit()



In [ ]:
async def main():
    await check_table_structure()

if __name__ == "__main__":
    import asyncio
    asyncio.run(main())


In [ ]:
import asyncio
import logging
from aiogram import Bot, Dispatcher, types
from aiogram.filters.command import Command
from aiogram.utils.keyboard import InlineKeyboardBuilder, ReplyKeyboardBuilder
from aiogram import F

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(name)s: %(message)s",
    force=True
)


# Объект бота
bot = Bot(token=API_TOKEN)
# Диспетчер
dp = Dispatcher()

def generate_options_keyboard(answer_options, right_answer):
    builder = InlineKeyboardBuilder()
    for i, option in enumerate(answer_options):
        # Используем индекс для callback_data
        callback_data = f"answer_{i}"  # Только индекс ответа
        builder.add(types.InlineKeyboardButton(
            text=option,
            callback_data=callback_data  # Передаем индекс ответа
        ))
    builder.adjust(1)
    return builder.as_markup()

@dp.callback_query(F.data)
async def answer_callback(callback: types.CallbackQuery):


    # Получаем текущий вопрос из базы данных
    current_question_index = await get_quiz_index(callback.from_user.id)
    correct_option = quiz_data[current_question_index]['correct_option']  # Индекс правильного ответа
    answer_index = int(callback.data.split("_")[1])  # Получаем индекс ответа из callback_data

    answer = quiz_data[current_question_index]['options'][answer_index]  # Текст выбранного ответа

    # Проверяем, правильный ли это ответ
    if answer_index == correct_option:
        await callback.message.answer(f"Вы выбрали: {answer} 🤔")
        await asyncio.sleep(1)
        await callback.message.answer("Правильно! 🎉😊")
        gif_url = 'https://i.pinimg.com/originals/5b/19/c7/5b19c73485c4010b7b05d7b1428c9192.gif'  # Прямая ссылка на файл GIF
        await callback.message.answer_animation(gif_url)
        current_score = await get_quiz_score(callback.from_user.id)
        current_score += 1  # Увеличиваем балл за правильный ответ
        await update_score(callback.from_user.id, current_score)
        await asyncio.sleep(1)
    else:
        await callback.message.answer(f"Вы выбрали: {answer}")
        await asyncio.sleep(1)
        correct_answer = quiz_data[current_question_index]['options'][correct_option]
        await callback.message.answer(f"Неправильно. 😭 Правильный ответ: {correct_answer}")
        await asyncio.sleep(1)

    # Переходим к следующему вопросу или завершаем квиз
    current_question_index += 1
    await update_quiz_index(callback.from_user.id, current_question_index)

    if current_question_index < len(quiz_data):
        await get_question(callback.message, callback.from_user.id)
    else:
        await callback.message.answer("Это был последний вопрос. Квиз завершен!")
        current_score = await get_quiz_score(callback.from_user.id)
        await callback.message.answer(f"Ваш результат: {current_score} ")
        gif_url = 'https://i.pinimg.com/originals/5b/19/c7/5b19c73485c4010b7b05d7b1428c9192.gif'  # Прямая ссылка на файл GIF
        await callback.message.answer_animation(gif_url)

@dp.message(Command("start"))
async def cmd_start(message: types.Message):
    builder = ReplyKeyboardBuilder()
    builder.add(types.KeyboardButton(text="Начать игру"))
    await message.answer("Добро пожаловать в квиз боби!", reply_markup=builder.as_markup(resize_keyboard=True))

async def get_question(message, user_id):

    current_question_index = await get_quiz_index(user_id)
    correct_index = quiz_data[current_question_index]['correct_option']
    opts = quiz_data[current_question_index]['options']
    kb = generate_options_keyboard(opts, opts[correct_index])
    number_of_questions = len(quiz_data)

    # Увеличиваем n для следующего вопроса
    await message.answer(f"Вопрос {current_question_index + 1} из {number_of_questions}: {quiz_data[current_question_index]['question']}", reply_markup=kb)

async def new_quiz(message):
    user_id = message.from_user.id
    current_question_index = 0
    await update_quiz_index(user_id, current_question_index)
    await get_question(message, user_id)

@dp.message(F.text == "Начать игру")
@dp.message(Command("quiz"))
async def cmd_quiz(message: types.Message):
    await message.answer(f"Давайте начнем квиз!")
    await new_quiz(message)

async def main():
    await create_table()
    await dp.start_polling(bot)

if __name__ == "__main__":
    asyncio.run(main())
